In [1]:
# Requirement: 'https://www.kaggle.com/c/fake-news/'

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score, classification_report


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('dataset/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
# Check NaN values
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
# drop the records with NaN values
df = df.dropna()

In [6]:
# Prepare the X nd Y set
X = df.iloc[:,:-1]
Y = df.label

### Preprocess the input

In [8]:
# Vocab size
vocab_size = 5000

In [9]:
messages = X.copy()
messages.reset_index(inplace=True)

#### 1. Stop words removal & stemming

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/himanshubhusan.rath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Function to prepare a corpus for the given feature
# Preprocessing like Stop word removal, non-characters removal, making to lower case etc are performed
def preprocess_text_input(data):
    
    ps = PorterStemmer()
    corpus = []
    
    for i in range(0, len(data)):
        # Remove all characters apart from a-z and A-Z
        text = re.sub('[^a-zA-Z]',' ',data[i])
        # Convert to lower case
        text = text.lower()
        # Get words from the sentences by splitting
        words = text.split()
        # filter stop words and perform stemming
        words = [ps.stem(word) for word in words if not word in stopwords.words('english')]
        # Join the words to form a sentence again
        text = ' '.join(words)

        corpus.append(text)
        
    return corpus


In [12]:
# preprocess the 'title' feature
corp = preprocess_text_input(messages['title'])
corp

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

#### 2. One-hot representation

In [16]:
onehot_repr = [one_hot(words, vocab_size) for words in corp]
onehot_repr

[[1329, 3680, 1510, 4891, 1104, 3951, 3920, 1312, 3799, 2623],
 [4021, 3374, 4041, 2442, 417, 1959, 3404],
 [2880, 1431, 1479, 217],
 [4791, 4074, 4191, 2081, 193, 4089],
 [4181, 417, 4684, 574, 621, 3542, 417, 1336, 182, 3905],
 [2606,
  1878,
  2472,
  3295,
  3195,
  2777,
  744,
  4790,
  3423,
  963,
  118,
  2889,
  2736,
  4308,
  3404],
 [28, 3983, 3337, 2919, 1137, 4270, 3522, 2974, 1231, 3555, 1413],
 [3483, 4424, 2895, 406, 4504, 4737, 2777, 2701, 1231, 3555, 1413],
 [1916, 3765, 2305, 4239, 2917, 4829, 2827, 403, 2777, 4895],
 [2997, 4940, 2778, 4947, 2948, 1839, 4534, 1693],
 [2995, 2861, 3970, 4914, 755, 4004, 3466, 2531, 4961, 4980, 2830],
 [2081, 1754, 1104, 4829, 2777, 4504],
 [4289, 3656, 2125, 3072, 2300, 1139, 760, 3548, 2156],
 [1205, 1224, 4104, 4436, 4540, 611, 1909, 1231, 3555, 1413],
 [3078, 1232, 2820, 1753, 4844, 1231, 3555, 1413],
 [4559, 1979, 772, 3907, 4085, 2663, 3765, 2730, 3887, 1938],
 [1290, 2397, 3374],
 [1399, 1416, 4182, 4087, 2777, 1464, 4271, 34

#### 3. Sequence padding

In [17]:
# Step-1 : Padding sequences to make all sentences equal in number of words
max_sentence_len = 20
embeded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=max_sentence_len)
embeded_docs.shape

(18285, 20)

## Define the model

In [19]:
embedding_feature_count = 40

model = Sequential()
# Add the embedding layer
model.add(Embedding(vocab_size, embedding_feature_count, input_length=max_sentence_len))
# Add LSTM layer (one layer with 100 LSTM cells) -- CHECK THIS
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


## Model training

In [20]:
# Prepare the training data
X_final = np.array(embeded_docs)
Y_final = np.array(Y)

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [22]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 29s 2ms/sample - loss: 0.2945 - accuracy: 0.8630 - val_loss: 0.2111 - val_accuracy: 0.9150
Epoch 2/10
12250/12250 [==============================] - 26s 2ms/sample - loss: 0.1358 - accuracy: 0.9475 - val_loss: 0.2087 - val_accuracy: 0.9157
Epoch 3/10
12250/12250 [==============================] - 25s 2ms/sample - loss: 0.0940 - accuracy: 0.9673 - val_loss: 0.2365 - val_accuracy: 0.9143
Epoch 4/10
12250/12250 [==============================] - 26s 2ms/sample - loss: 0.0668 - accuracy: 0.9773 - val_loss: 0.2852 - val_accuracy: 0.9112
Epoch 5/10
12250/12250 [==============================] - 31s 3ms/sample - loss: 0.0446 - accuracy: 0.9854 - val_loss: 0.3446 - val_accuracy: 0.9130
Epoch 6/10
12250/12250 [==============================] - 23s 2ms/sample - loss: 0.0239 - accuracy: 0.9927 - val_loss: 0.4394 - val_accuracy: 0.9099
Epoch 7/10
12250/12250 [==============================] -

## Performance Metrics

In [24]:
pred = model.predict_classes(X_test)

In [25]:
confusion_matrix(pred,Y_test)

array([[3115,  242],
       [ 304, 2374]])

In [26]:
accuracy_score(pred, Y_test)

0.9095277547638774

In [30]:
print(classification_report(Y_test,pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [32]:
print(classification_report(y_true=Y_test, y_pred=pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

